## Проверка данных о посещении врачей, представленных ФОМС

In [1]:
import pandas as pd
import numpy as np
import os
import random

# игнорирование предупреждения о формате при импорте из excel
import warnings

warnings.simplefilter("ignore")

In [2]:
df_test = pd.read_csv('C:/work/med/test/52000.csv', sep=';', dtype=object, header=None)

In [3]:
df_test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2023,1,NaN,555508,1.2.643.5.1.13.13.12.2.55.5446,55550800,55080007,46,65,3,5500000001130322,H10.9,1.1,NaN,0.00
1,2023,1,NaN,555508,1.2.643.5.1.13.13.12.2.55.5446,55550800,55080007,46,65,3,5500000001066333,T15.0,1.1,NaN,0.00
2,2023,1,52000,555508,1.2.643.5.1.13.13.12.2.55.5446,55550801,55080101,46,65,3,5500000001065937,H40.1,1.0,1,0.00
3,2023,1,52000,555508,1.2.643.5.1.13.13.12.2.55.5446,55550801,55080101,46,65,3,5500000001131483,H26.2,1.0,1,0.00
4,2023,1,52000,555508,1.2.643.5.1.13.13.12.2.55.5446,55550801,55080101,46,65,3,5500000001065937,H40.1,1.0,1,0.00


In [4]:
# проверка количества столбцов
e_0 = "\t- число столбцов "+str(df_test.shape[1])+" не соответствует модели" if df_test.shape[1]!=15 else ""


In [5]:
# проверка столбца 9 на "3"
e_test_3_1=""
try:
    e_test_3="\t- не все коды условий оказания МП равны 3" if df_test[9].unique().astype("int").sum()!=3 else ""
except Exception:
    e_test_3_1="\t- в кодах условий оказания МП присутствуют нечисловые или нецелые значения"

In [6]:
# правильные названия столбцов
dict_columns={0: "year",
             1: "monht",
             2: "oktmo",
             3: "id_mo",
             4: "id_oid",
             5: "id_d",
             6: "id_r",
             7: "id_spec",
             8: "id_pr",
             9: "id_ump",
             10: "id_doc",
             11: "id_ds",
             12: "id_pc",
             13: "v_count",
             14: "yet_sum",}

In [7]:
# проверка пустых значений
for column in df_test.columns:
    if df_test[column].isnull().sum()!=0:
        print(f"\t- столбец {dict_columns[column]}:\tпустых значений {df_test[column].isnull().sum()}")

	- столбец oktmo:	пустых значений 2
	- столбец v_count:	пустых значений 2


In [8]:
df_test[9] = df_test[9].astype('int')

In [9]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2572558 entries, 0 to 2572557
Data columns (total 15 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   0       object
 1   1       object
 2   2       object
 3   3       object
 4   4       object
 5   5       object
 6   6       object
 7   7       object
 8   8       object
 9   9       int32 
 10  10      object
 11  11      object
 12  12      object
 13  13      object
 14  14      object
dtypes: int32(1), object(14)
memory usage: 284.6+ MB


In [10]:
# генератор индексов для случайного перебора строк в наборе
rate_n=0.03
test_list = random.sample(range(0, df_test.shape[0]), int(df_test.shape[0]*rate_n))

In [11]:
print(len(test_list) == len(set(test_list)))

True


In [13]:
%%time

for test_index in test_list:
    e_oid=""
    if len(df_test.loc[test_index][4].split("."))!=11:
        print(test_index, df_test.loc[test_index][4], "неверное число групп")
        e_oid = "нарушение структуры oid"
    e_year=""
    if df_test.loc[test_index][0] not in ["2022", "2023"]:
        print(f"нарушение целостности данных в строке {test_index}: год = ", df_test.loc[test_index][0])
        e_year = "нарушение диапазона значений года"
    e_month=""
    if df_test.loc[test_index][1] not in ["1",
                                          "01",
                                          "2",
                                          "02",
                                          "3",
                                          "03",
                                          "4",
                                          "04",
                                          "5",
                                          "05",
                                          "6",
                                          "06",
                                          "7",
                                          "07",
                                          "8",
                                          "08",
                                          "9",
                                          "09",
                                          "10",
                                          "11",
                                          "12"]:
        print(f"нарушение целостности данных в строке {test_index}: месяц = ", df_test.loc[test_index][1])
        e_month = "нарушение диапазона значений месяца"
    e_oktmo=""
    if len(df_test.loc[test_index][3])!=11:
        e_oktmo="\t- обнаружены нарушения значений кода ОКТМО: количество символов не равно 11"
print(f"На тестовой выборке N={rate_n*100}% от набора данных:")
if e_year=="":
    print("\t- не обнаружено нарушений диапазона значений года")
else:
    print("\t- обнаружены нарушения диапазина значенй года")
if e_month=="":
    print("\t- не обнаружено нарушений диапазона значений месяца")
else:
    print("\t- обнаружены нарушения диапазона значений месяца")
if e_oid=="":
    print("\t- не обнаружено нарушений структуры oid")
else:
    print("\t- обнаружены нарушения структуры oid")
if e_oktmo=="":
    print("\t- не обнаружено нарушений структуры кода ОКТМО")
else:
    print(e_oktmo)

На тестовой выборке N=3.0% от набора данных:
	- не обнаружено нарушений диапазона значений года
	- не обнаружено нарушений диапазона значений месяца
	- не обнаружено нарушений структуры oid
	- обнаружены нарушения значений кода ОКТМО: количество символов не равно 11
CPU times: total: 6.47 s
Wall time: 12.7 s
